<a href="https://colab.research.google.com/github/joohwan38/DeepRL-Study/blob/main/planegame_RL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install stable_baselines3
!pip install gym
!pip install pygame

     |████████████████████████████████| 172 kB 34.9 MB/s 
     |████████████████████████████████| 11.8 MB 13.8 MB/s 


In [3]:
#import GYM stuff
import gym 
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete 

#import helpers
import numpy as np
import random
import os

#import Stable baselines stuff
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

#import pygame
import pygame
import time
from datetime import datetime

# pygame.quit()

pygame 2.0.1 (SDL 2.0.14, Python 3.7.11)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
pygame.init()
#오브젝트 좌표, 이속, 사이즈 초기값 세팅
class obj:
    def __init__(self):
        self.x = 0
        self.y = 0
        self.move = 0
    def put_img(self, a, b):
        self.sx, self.sy = a, b

#충돌 판정 (y/n)
def crash(a, b):
    if (a.x-b.sx <= b.x) and (b.x <= a.x+a.sx):
        if (a.y-b.sy <= b.y) and (b.y <= a.y+a.sy):
            return True
        else:
            return False
    else:
        return False

#커스텀 환경 세팅)

class PlaneEnv(Env):
    
    def __init__(self):
        # Actions 조작의 범위 설정
        self.action_space = Discrete(3) 
        # 옵저베이션 설정 (x축, y축)
        self.observation_space = MultiDiscrete([500, 500, 900, 500, 900, 500, 900, 500, 900, 500, 900])
        # 시작 값 설정 (all zero)
        self.state = np.zeros(11, dtype=int)
        
        self.ss = obj()
        #비행선 사이즈
        self.ss.put_img(50, 60)
        
        #비행선 시작 좌표
        self.ss.x = 205 #round(size[0]/2 - self.ss.sx/2)
        self.ss.y = 825 #size[1] -self.ss.sy - 15
        self.ss.move = 20
        
        
        
    def step(self, action):
        global done
        
        reward =2
        
        if action ==0:
            self.ss.x -= self.ss.move
            if self.ss.x <= 50:
                self.ss.x = 50
        elif action ==1:
            self.ss.x += self.ss.move
            if self.ss.x >= 425:
                self.ss.x = 425
        elif action ==2:
            pass
                
        if random.random() > 0.90:
            aa = obj()
            aa.put_img(40, 40)
            mylist = [self.ss.x, random.randrange(0,478)]
            aa.x = random.choices(mylist, weights = [10, 2], k = 1)[0]
            aa.y = 10
            aa.move = 25
            a_list.append(aa)
            
        for i in range(len(a_list)):
            if a_list[i].y < 825:
                pass
            elif (abs(self.ss.x+25 - a_list[i].x+20) * abs(self.ss.y+30 - a_list[i].y+20))**(1/2) < 100:
                reward -= round((abs(self.ss.x+25 - a_list[i].x+20) * abs(self.ss.y+30 - a_list[i].y+20))**(1/2)/10)
                
        for i in range(len(a_list)):
            a = a_list[i]
            if crash(a, self.ss) == True:
                done=True
                reward -=400
        
        self.state[0] = self.ss.x
        for i in range(len(a_list[:(len(self.state)-1)//2])):
            self.state[2*i+1] = a_list[i].x
            self.state[2*i+2] = a_list[i].y

        d_list = []
        for i in range(len(a_list)):
            ea = a_list[i]
            ea.y += ea.move
            if ea.y >= size[1]:
                d_list.append(i)
                reward +=30
        for d in d_list:
            del a_list[d]
        
        info = {}
        
        # Return step information
        return self.state, reward, done, info



    def render(self):
        pass
    
    
    def reset(self):
        # Reset shower temperature
        self.state = np.zeros(11, dtype=int)
        # Set shower length
        self.shower_length = 220
        self.ss = obj()
        self.ss.put_img(50, 60)
        self.ss.x = 205 #round(size[0]/2 - self.ss.sx/2)
        self.ss.y = 825 #size[1] -self.ss.sy - 15
        self.ss.move = 20
        
        return self.state

In [5]:
size = [500, 900]

env=PlaneEnv()

a_list=[]
d_list=[]


In [6]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0
    reward = 0
    a_list=[]
    d_list=[]
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
#         print(action, n_state, score, done)
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:-221
Episode:2 Score:-178
Episode:3 Score:-273
Episode:4 Score:-324
Episode:5 Score:-191
Episode:6 Score:-258
Episode:7 Score:-149
Episode:8 Score:-264
Episode:9 Score:-322
Episode:10 Score:-302


In [7]:
log_path = os.path.join('Training', 'Logs')

In [8]:
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [ ]:
model.learn(total_timesteps=10000000)#, callback=eval_callback)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
|    value_loss           | 6.61e+04      |
-------------------------------------------
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 1             |
|    ep_rew_mean          | -200          |
| time/                   |               |
|    fps                  | 220           |
|    iterations           | 1887          |
|    time_elapsed         | 17498         |
|    total_timesteps      | 3864576       |
| train/                  |               |
|    approx_kl            | 0.00042938054 |
|    clip_fraction        | 0.00879       |
|    clip_range           | 0.2           |
|    entropy_loss         | -0.0984       |
|    explained_variance   | 0             |
|    learning_rate        | 0.0003        |
|    loss                 | 4.67e+04      |
|    n_updates            | 18860         |
|    policy_gradient_loss | -0.00084      |
|    value_loss           | 6.34e+04    

In [ ]:
plane_path = os.path.join('Training', 'Saved Models', 'evolution_10M_PPO')

In [ ]:
model.save(plane_path)
evaluate_policy(model, env, n_eval_episodes=100, render=False)

In [ ]:
# training_log_path = os.path.join(log_path, 'PPO_20')

In [ ]:
model = PPO.load(plane_path, env)

In [ ]:
s_list=[]

episodes = 10
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0
    reward = 0
    a_list=[]
    d_list=[]
    
    k=0
    while not done:
        k+=1
        env.render()
        action, _states = model.predict(obs)
        obs, reward, done, info = env.step(action)
        score+=reward
        if k %10 == 0:
          print(obs, action, score)
    s_list.append(k)
    print('Episode:{} Score:{} Total Steps:{}'.format(episode, score, k))
print('mean K:{}'.format(np.mean(s_list)))
env.close()

In [ ]:
s_list=[]

episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0
    reward = 0
    a_list=[]
    d_list=[]
    
    k=0
    while not done:
        k+=1
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
        # if k %10 == 0:
        #   print(obs, action, score)
    s_list.append(k)
    print('Episode:{} Score:{} Total Steps:{}'.format(episode, score, k))
print('mean K:{}'.format(np.mean(s_list)))
env.close()